In [1]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams
from packages.Model.CFNet import CFNet1_1

Using TensorFlow backend.


# Inisialisasi / Persiapan

In [2]:
fold_index = 0
dataset_dir = 'dataset/bin/CF.32.B.30K'
trained_dir = 'trained/CF.32.B.30K CF1.1.2'

# Memuat Dataset

In [3]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [4]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Parameter Gabor

In [5]:
class ChannelizeGaborParams(GaborParams):
    
    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = multiplier * i
        
        i += 1
        j += 1
        
        lambd = (i * j) / shape[2]
        theta = degree * np.pi / 180
        psi = (i + j) / shape[2]
        sigma = (i + j)
        gamma = (i + j) / shape[2]
        
        return lambd, theta, psi, sigma, gamma

# Mendefinisikan Kernel Gabor

In [6]:
kernel1 = GaborFilterBanks(ChannelizeGaborParams())
kernel2 = GaborFilterBanks(ChannelizeGaborParams())
kernel3 = GaborFilterBanks(ChannelizeGaborParams())
kernel4 = GaborFilterBanks(ChannelizeGaborParams())
kernel5 = GaborFilterBanks(ChannelizeGaborParams())
kernel6 = GaborFilterBanks(ChannelizeGaborParams())

# Mendefinisikan Model

In [9]:
ars = CFNet1_1(kernels = {
    1: kernel1,
    2: kernel2,
    3: kernel3,
    4: kernel4,
    5: kernel5,
    6: kernel6
}, loss = 'categorical_crossentropy', optimizer = 'adadelta')
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')

model = ars.model
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
c2 (Conv2D)                  (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
c3 (Conv2D)                  (None, 16, 16, 64)        18496     
_________________________________________________________________
batc

# Pelatihan

In [ ]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 100, verbose = 1, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch = 0)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
22500/22500 [==============================] - 288s 13ms/step - loss: 2.0749 - acc: 0.2907
New best accuracy : 0.3217333333333333
Model saved to disk
Passes : 0/50
Epoch 2/100
22500/22500 [==============================] - 342s 15ms/step - loss: 1.8840 - acc: 0.3368
New best accuracy : 0.3547111111111111
Model saved to disk
Passes : 0/50
Epoch 3/100
22500/22500 [==============================] - 353s 16ms/step - loss: 1.8095 - acc: 0.3592
New best accuracy : 0.3566666666666667
Model saved to disk
Passes : 0/50
Epoch 4/100
22500/22500 [==============================] - 337s 15ms/step - loss: 1.7552 - acc: 0.3735
New best accuracy : 0.38084444444444443
Model saved to disk
Passes : 0/50
Epoch 5/100
22500/22500 [==============================] - 339s 15ms/step - loss: 1.7231 - acc: 0.3853
New best accuracy : 0.39244444444444443
Model saved to disk
Passes : 0/50
Epoch 6/100
22500/22500 [==============================] - 338s 15ms/s